In [ ]:
import importlib
from utils import dataset_utils
importlib.reload(dataset_utils)

def infer_all_cases():
    cases = ["A", "A_B", "B", "B_A", "AB"] #["A", "A_B", "B", "B_A", "AB"]
    all_metrics = {}

    for case in cases:
        #case = "A" #A, A_B, B, B_A, AB
        if case == "A" or case == "A_B":
            """
            folders_list = [
                "fold0_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-m_Aug19_09-40-29",
                "fold1_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-m_Aug19_10-15-21",
                "fold2_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-m_Aug19_10-50-28",
                "fold3_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-m_Aug19_09-42-53",
                "fold4_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-m_Aug19_10-18-34"
            ]
            """
            folders_list = [
                "fold0_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-mn_Aug22_14-05-43",
                "fold1_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-mn_Aug22_14-59-16",
                "fold2_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-mn_Aug22_15-53-18",
                "fold3_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-mn_Aug22_16-47-39",
                "fold4_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-mn_Aug22_17-42-00"
            ]
        elif case == "B" or case == "B_A":
            folders_list = [
                "fold0_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-n_Aug19_14-13-35",
                "fold1_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-n_Aug19_14-45-29",
                "fold2_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-n_Aug19_15-18-20",
                "fold3_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-n_Aug19_15-51-21",
                "fold4_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-n_Aug19_16-23-56"
            ]
        elif case == "AB":
            folders_list = [
                "fold0_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-mn_Aug20_14-21-30",
                "fold1_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-mn_Aug20_16-20-40",
                "fold2_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-mn_Aug20_18-21-17",
                "fold3_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-mn_Aug20_20-22-07",
                "fold4_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-mn_Aug20_22-22-35"
            ]

        y_all = []
        y_pred_all = []
        y_prob_all = []

        metrics = pd.DataFrame({"model": [], "AUC": [], "acc": [], "spec": [], "sens": [], "prec": []})
        i = 0

        for folder in folders_list:
            model_common_path = "../RSNA-BTC-Datasets/out_models/"
            model_folder = model_common_path + folder

            model_basefile, info = get_best_model(model_folder)
            m_modelfile = model_folder + "/" + model_basefile
            transform = None
            size = len(info["mri_types"])
            sel_slices = 1 if info["dims"] == 2 else None
            chosen_net = "alt" if info["dims"] == 2 else "sim"

            m_model = try_and_get_model(chosen_net, info["dims"], size, info["is_depth_wise"], info["output_size"])

            #m_modelfile = m_modelfile_3d
            print(m_modelfile)
            print(info)
            print(f"-- FOLD N:{i}")

            if case == "A":
                to = "m"#info["train_origin"]
                data_idx = 1
            elif case == "A_B":
                to = "n"
                data_idx = 2 #full
            elif case == "B":
                to = "n"
                data_idx = 1
            elif case == "B_A":
                to = "m"
                data_idx = 2 #full
            elif case == "AB":
                to = "mn"
                data_idx = 1
            print(f"Train origin: {to}")
            if to == "m":
                packs = dataset_utils.generate_datasets(['KLF'], info, transform, sel_slices, m_path=dir_path) #m_path=dir_path, n_path=new_dir_path
            elif to == "n":
                packs = dataset_utils.generate_datasets(['KLF'], info, transform, sel_slices, n_path=new_dir_path)
            elif to == "mn":
                packs = dataset_utils.generate_datasets(['KLF'], info, transform, sel_slices, m_path=dir_path, n_path=new_dir_path)
            print(packs)
            loader_packs = dataset_utils.get_loaders(packs, info, is_fold=True, fold_num=i)
            print(loader_packs)

            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            logging.info("Using CUDA...")

            pd.set_option("display.max_rows", None, "display.max_columns", None)
            #print(loader_packs['KLF']['m_dataloaders'])
            selected_type = info["mri_types"][0]
            print(selected_type)
            dataloader = loader_packs[selected_type][f"{to}_dataloaders"]#m_dataloaders[0]
            val_loader = dataloader[data_idx]
            #val_loader = dataloader[2] #full

            print("\nPREDICTIONS:")
            print(folder)
            #print(f"Test set: {to} (B)")
            test_loader = val_loader
            m_test_pred, ids, X, y, y_pred, y_prob = predict(m_model, device, m_modelfile, test_loader, size)
            pd_metrics = get_metrics(y, y_pred, y_prob, f"{folder}_{case}")
            y_all = y_all + y
            y_pred_all = y_pred_all + y_pred
            y_prob_all = y_prob_all + y_prob

            preddfA = pd.DataFrame({"BraTS21ID": ids, "MGMT_real_value": y, "MGMT_pred_value": y_pred, "MGMT_prob_value": y_prob}) 
            preddfA = preddfA.sort_values(by="BraTS21ID")
            preddfA.to_csv(f"pred_metrics/{folder}_{case}.csv", index=False)
            metrics = metrics.append(pd_metrics, ignore_index=True)
            #pd_metrics.to_csv(f"pred_metrics/{folder}_{to}_A_metrics.csv", index=False)

            print("----------\n")

            i += 1

        pd_metrics = get_metrics(y_all, y_pred_all, y_prob_all, f"all_{case}")
        metrics = metrics.append(pd_metrics, ignore_index=True)
        metrics.to_csv(f"pred_metrics/all_metrics_fold_2d_{case}.csv", index=False)
        all_metrics[case] = metrics
    return all_metrics

In [ ]:
def mix_infers():
    csv_list = []
    cases = ["A", "A_B", "B", "B_A", "AB"] #["A", "A_B", "B", "B_A", "AB"]
    all_metrics = {}
    #case = "A"
    sizes = ["2D", "3D", "Tun"]
    x_all_dict = {}
    y_all_dict = {}
    y_pred_all_dict = {}
    y_prob_all_dict = {}
    pd_all_metrics = pd.DataFrame({"model": [], "AUC": [], "acc": [], "spec": [], "sens": [], "prec": []})
    pd_all_metrics_mag = pd.DataFrame({"model": [], "-": [], "acc": [], "spec": [], "sens": [], "prec": []})

    for case in cases:
        for size in sizes:
            #case = "A" #A, A_B, B, B_A, AB
            if case == "A" or case == "A_B":
                """
                folders_list = [
                    "fold0_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-m_Aug19_09-40-29",
                    "fold1_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-m_Aug19_10-15-21",
                    "fold2_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-m_Aug19_10-50-28",
                    "fold3_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-m_Aug19_09-42-53",
                    "fold4_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-m_Aug19_10-18-34"
                ]
                """
                if size == "2D":
                    csv_list = [
                        "fold0_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-mn_Aug22_14-05-43",
                        "fold1_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-mn_Aug22_14-59-16",
                        "fold2_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-mn_Aug22_15-53-18",
                        "fold3_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-mn_Aug22_16-47-39",
                        "fold4_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-mn_Aug22_17-42-00"
                    ]
                elif size == "3D":
                    csv_list = [
                        "fold0_RSNAClassifierSingle3D-DW-SO-KLF-mn_Aug22_13-59-17",
                        "fold1_RSNAClassifierSingle3D-DW-SO-KLF-mn_Aug22_14-52-37",
                        "fold2_RSNAClassifierSingle3D-DW-SO-KLF-mn_Aug22_15-46-27",
                        "fold3_RSNAClassifierSingle3D-DW-SO-KLF-mn_Aug22_16-38-47",
                        "fold4_RSNAClassifierSingle3D-DW-SO-KLF-mn_Aug22_17-31-05"
                    ]
                else:
                    csv_list = [
                        "tunisiaai_A"
                    ]
            elif case == "B" or case == "B_A":
                if size == "2D":
                    csv_list = [
                        "fold0_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-n_Aug19_14-13-35",
                        "fold1_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-n_Aug19_14-45-29",
                        "fold2_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-n_Aug19_15-18-20",
                        "fold3_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-n_Aug19_15-51-21",
                        "fold4_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-n_Aug19_16-23-56"
                    ]
                elif size == "3D":
                    csv_list = [
                        "fold0_RSNAClassifierSingle3D-DW-SO-KLF-n_Aug19_14-11-53",
                        "fold1_RSNAClassifierSingle3D-DW-SO-KLF-n_Aug19_14-43-40",
                        "fold2_RSNAClassifierSingle3D-DW-SO-KLF-n_Aug19_15-16-05",
                        "fold3_RSNAClassifierSingle3D-DW-SO-KLF-n_Aug19_15-48-34",
                        "fold4_RSNAClassifierSingle3D-DW-SO-KLF-n_Aug19_16-20-47"
                    ]
                else:
                    csv_list = [
                        "tunisiaai_B"
                    ]
            elif case == "AB":
                if size == "2D":
                    csv_list = [
                        "fold0_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-mn_Aug20_14-21-30",
                        "fold1_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-mn_Aug20_16-20-40",
                        "fold2_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-mn_Aug20_18-21-17",
                        "fold3_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-mn_Aug20_20-22-07",
                        "fold4_RSNAAlternativeClassifierSingle2D-DW-SO-KLF-mn_Aug20_22-22-35"
                    ]
                elif size == "3D":
                    csv_list = [
                        "fold0_RSNAClassifierSingle3D-DW-SO-KLF-mn_Aug20_13-42-35",
                        "fold1_RSNAClassifierSingle3D-DW-SO-KLF-mn_Aug20_15-38-50",
                        "fold2_RSNAClassifierSingle3D-DW-SO-KLF-mn_Aug20_17-37-56",
                        "fold3_RSNAClassifierSingle3D-DW-SO-KLF-mn_Aug20_19-36-30",
                        "fold4_RSNAClassifierSingle3D-DW-SO-KLF-mn_Aug20_21-33-35"
                    ]
                else:
                    csv_list = [
                        "tunisiaai_AB"
                    ]

            x_all = []
            y_all = []
            y_pred_all = []
            y_prob_all = []
            i = 0

            for c in csv_list:
                res = pd.read_csv(f"pred_metrics/{c}_{case}.csv")
                x = list(res["BraTS21ID"])
                y = list(res["MGMT_real_value"])
                y_pred = list(res["MGMT_pred_value"])
                y_prob = list(res["MGMT_prob_value"])
                #print(i)
                #metrics = pd.DataFrame({"model": [], "AUC": [], "acc": [], "spec": [], "sens": [], "prec": []})
                i += 1

                x_all.extend(x)
                y_all.extend(y)
                y_pred_all.extend(y_pred)
                y_prob_all.extend(y_prob)

            #print(case, size)
            #print(y_prob_all)
            x_all_dict[size] = x_all
            y_all_dict[size] = y_all
            y_pred_all_dict[size] = y_pred_all
            y_prob_all_dict[size] = y_prob_all

        y_prob_avg = [ (float(two[1:-1])+float(three[1:-1]))/2 for two, three in zip(y_prob_all_dict["2D"], y_prob_all_dict["3D"]) ]
        #print(f"{case} AVG")
        #print(y_prob_avg)
        y_pred_avg = [ int(elem>0.5) for elem in y_prob_avg ]
        pd_metrics = get_metrics(y_all_dict["2D"], y_pred_avg, y_prob_avg, f"AVG_{case}")
        #print(pd_metrics)
        pd_all_metrics = pd_all_metrics.append(pd_metrics, ignore_index=True)
        
        y_pred_mag = [ int(float(two)+float(three)+float(tun) > 1.5) for two, three, tun in zip(y_pred_all_dict["2D"], y_pred_all_dict["3D"], y_pred_all_dict["Tun"]) ]
        pd_metrics_mag = get_metrics(y_all_dict["2D"], y_pred_mag, y_prob_avg, f"MAG_{case}")
        pd_all_metrics_mag = pd_all_metrics_mag.append(pd_metrics_mag, ignore_index=True)
    print(pd_all_metrics)
    print(pd_all_metrics_mag)
    #print(x_all_dict)
    #print(y_all_dict)
    #print(y_pred_all_dict)
    #final_out = = pd.DataFrame({"X": [], "Y": [], "Y_pred_A": [], "Y": [], "sens": [], "prec": []})
    

In [ ]:
mix_infers()